# Training a good double DQN model and testing it

In [1]:
import sys
import os

from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam
from rl.agents.dqn import DQNAgent, Sequential
from rl.memory import SequentialMemory
from rl.policy import EpsGreedyQPolicy, LinearAnnealedPolicy, BoltzmannQPolicy
from rl.callbacks import FileLogger

# add local path
sys.path.append(os.path.join(os.getcwd(), "./src"))

from rl_model import TorcsKerasTrainer

/home/nikitautiu/envs/mirpr/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Model building
We are building a double DQN model identical to the one used in training.

In [2]:
def build_model(obs_shape, nb_actions):
    # Next, we build a very simple model.
    model = Sequential()
    model.add(Flatten(input_shape=(1,) + obs_shape))
    model.add(Dense(35))
    model.add(Activation('relu'))
    model.add(Dense(25))
    model.add(Activation('relu'))
    model.add(Dense(nb_actions))
    model.add(Activation('linear'))
    print(model.summary())

    # Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
    # even the metrics!
    memory = SequentialMemory(limit=500000, window_length=1)

    # deep q learning agent, uses greedy-epsilon policy, does NB_STEPS steps of warmup and
    # updates the gradient every 1000 steps
    dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=30000,
                   target_model_update=1e-2, policy=EpsGreedyQPolicy(0.1), enable_double_dqn=True)
    dqn.compile(Adam(lr=1e-3), metrics=['mae'])
    return dqn

In [5]:
dqn = TorcsKerasTrainer(throttle=True, discrete_actions=9, model_function=build_model,
                            obs_fields=['speedX', 'speedY', 'speedZ', 'track', 'focus', 'trackPos', 'angle'])
# saving best model fo reach on of the runs
# laos log every episode result in a separate file for later use
dqn.fit(nb_steps=500000, best_filename='best_double_dqn.h5f', verbose=1,
        callbacks=[FileLogger('double_dqn.json')])

dqn.save_weights('double_dqn.h5f')

## Test run
Now that we have the trained model, we can test it.

In [ ]:
dqn.load_weights('double_dqn.h5f')
dqn.test(nb_episodes=1)